# Isolation Forest

## Load Data

In [ ]:
## Libraries
# lightkurve
#import lightkurve as lk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
# import warnings
# import pickle
from sklearn.ensemble import IsolationForest
# from sklearn.preprocessing import StandardScaler

In [10]:
##("../0.Data/031381302_imputed.csv")

## Data
df = pd.read_csv("../0.Data/031381302_imputed.csv", index_col = [0])

# Check
print(df.shape)
print(df.head())

## Time Series of interest
lightcurve = df.loc[:, ["pdcsap_clean"]]
lightcurve.sort_index(inplace=True)

# Check
print(lightcurve.shape); lightcurve.info(); lightcurve.head()

(18856, 4)
             pdcsap_flux  pdcsap_clean        trend  seasonal
1437.978646          NaN      1.423914  1564.788866 -1.753096
1437.980034          NaN     -1.225973  1564.784308  0.901349
1437.981423          NaN      1.204118  1564.779761 -1.524195
1437.982812          NaN      2.528983  1564.775225 -2.844523
1437.984201          NaN      0.485369  1564.770699 -0.796384
(18856, 1)
<class 'pandas.core.frame.DataFrame'>
Float64Index: 18856 entries, 1437.978645522663 to 1464.2854684532865
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pdcsap_clean  18856 non-null  float64
dtypes: float64(1)
memory usage: 294.6 KB


,pdcsap_clean
1437.978646,1.423914
1437.980034,-1.225973
1437.981423,1.204118
1437.982812,2.528983
1437.984201,0.485369


## Model Setup

In [ ]:
##

## Train Model

In [ ]:
## Isolation Forest
model = IsolationForest(contamination = "auto") #, random_state = 42)
# contamination: proportion of anomalies in the data.
# random_state: for reproducibility.
model.fit(lightcurve)

## Predict anomalies
anomalies = model.predict(lightcurve)

In [ ]:
## Plot - Highlight anomalies
plt.figure(figsize=(15, 6))
plt.plot(data['value'], label='Time Series', color='blue')
plt.scatter(data[data['anomaly'] == 1].index, 
            data[data['anomaly'] == 1]['value'], 
            color='red', label='Anomalies')
plt.title('Anomaly Detection using Isolation Forest')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

## Exploration

In [ ]:
##